# PDF 文書の内容にもとづいて回答するボットの作り方を理解する
## はじめに
生成 AI や大規模言語モデル （LLM）は、膨大な量のテキストデータのトレーニングを受けており、テキストを生成し、言語を翻訳し、さまざまな種類のクリエイティブ コンテンツを作成し、有益な方法で質問に答えることができます。ただし、LLM には、企業で使用する場合の問題点がいくつかあります。
その一つに、生成 AI は誤った情報を含むテキストを生成したり、誤った情報を翻訳したりする可能性があります。企業における生成 AI 利用では、これが問題となることがあります。

そこで生成 AI をエンジンとして利用しながら、企業内の信頼のおける情報の中から適切な回答してほしいというユースケースがあります。

これを実現する大きな処理の流れは次のようになっています。

1.   PDF ファイルを読み込む
2.   ページごとでもよいのですが、もう少し細かい単位にテキストを分割する
3.   分割したテキスト情報をエンべディング API を利用してベクトル化 / エンべディング
4.   ベクトル情報とテキスト情報の関連を保持する
5.   ユーザーの問い合わせ内容をベクトル化 / エンべディングする
6.   5. のベクトルと最も類似度の高いベクトルを複数個取得する
7.   6. のベクトルを生成元のテキスト情報を取得する
8.   複数個のテキスト情報をコンテキスト情報として与え、この情報の中から質問の回答を作成するように LLM に問い合わせをする
9.   PDF 文書の内容にもとづく回答が得られます

これを実現するためにエンべディング、ベクトル検索やコンテンツの保持といった技術要素が必要となりますが、Google Cloud では Vertex AI Search というソリューションを提供しているため、個々の技術要素をバラバラに組み合わせるのではなく、Google の強みである検索技術をとりこんだソリューションを活用して簡単にソリューションを構築することが可能です。

ここでは、生成 AI 利用時のフレームワークである LangChain を利用して、Vertex AI Search を利用して企業の信頼するデータソースから情報を取得し、その情報をもとにチャットボット型で質問に回答する流れを確認します。

## Vertex AI Search のセットアップ

次に、リンク先の手順に従って Vertex AI Search をセットアップします。

[Create and preview a search app for unstructured data from Cloud Storage](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_and_preview_a_search_app_for_unstructured_data_from)

パラメーターは以下を指定していることを確認してください。
- アプリケーションの種類: 検索
- インデックスするデータの種類: 非構造化データ( PDF や HTML )

必要に応じて、Cloud Storage にバケットを作成し、回答させたいコンテンツを含む PDF ファイルをアップロードしておきます。
このハンズオンでは、次のファイルをアップロードする前提で記載しています。
 - [Alphabet 2022 10K annual report](https://abc.xyz/assets/9a/bd/838c917c4b4ab21f94e84c3c2c65/goog-10-k-q4-2022.pdf)
 - [Google Cloud セキュリティ ホワイトペーパー](https://services.google.com/fh/files/misc/security_whitepapers_4_booklet_jp.pdf)
 - [「Google Cloud Day: Digital '22 - 15 のトピックから学ぶ」🌟eBook](https://lp.cloudplatformonline.com/rs/808-GJW-314/images/Google_ebooks_all_0614.pdf)

インデックスの作成にしばらく時間がかかります。Vertex AI Search のプレビュー機能で検索ができるか確認することが可能です。

最後に、作成した Search のアプリの「データ」ビューより、作成したデータストアの ID をメモしておきます。

## 環境セットアップ

python のバージョンを確認します。最新の LangChain は `Requires-Python >=3.8.1,<4.0`が前提となっています。

In [ ]:
import sys
print(sys.version)

前提パッケージを導入します。

In [ ]:
# Install Vertex AI LLM SDK
! pip install langchain langchain-google-vertexai langchain-google-community google-cloud-discoveryengine --upgrade --user

**※ 注意: ここでカーネルを再起動します。**

* Colab の場合、上記のログ、もしくは、ダイアログで "RESTART RUNTIME" ボタンが表示された場合、ボタンを押してカーネルをリスタートできます。
* Vertex AI Workbench の場合、メニューよりカーネルのリスタートを実行できます。

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

続いて、Google Cloud でプロジェクトを作成し Vertex AI API を有効化します。

また、このコードを実行するユーザーに`Vertex AI ユーザー`、`ディスカバリー エンジン閲覧者`のロールを付与します。

Colab の場合、以下を実行し Vertex AI API のユーザー権限をもつアカウントでログインします。 Vertex AI Workbench の場合はスキップされます。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

環境変数などを定義します。 Google Cloud のプロジェクト ID、Vertex AI Search の DATASTORE ID を指定してください。

In [ ]:
PROJECT_ID = "<your_project_id>"  # @param {type:"string"}
DATASTORE_ID = "<datastore_id>"  # @param {type:"string"}
REGION = "asia-northeast1"

Vertex AI と LangChain のライブラリーの導入を確認します。 LangChain v0.0.208 で動作確認しています。

In [ ]:
import langchain
from google.cloud import aiplatform

print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")

import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

## Vertex AI Gemini API の準備

LangChain を利用して Gemini API のText、Chat、Embeddings モデルを取得します。

In [ ]:
from langchain_google_vertexai import VertexAI

# Text model instance integrated with LangChain
llm = VertexAI(
    model_name="gemini-1.0-pro",
    max_output_tokens=2048,
    temperature=0.5,
    verbose=True,
)

ここでは、Vertex AI Search に最大3つの文書、１つの文書から3つの extractive answer を回答するように設定します。

※ RAG アプリケーションのコンテキスト情報として渡すチャンクに相当しますので、要件に応じて適宜調整してください。

In [ ]:
from langchain_google_community import VertexAISearchRetriever

# Vertex AI Search retriever
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID, data_store_id=DATASTORE_ID, get_extractive_answers=True, max_extractive_answer_count=3, max_documents=3
)

## Vertex AI Search に格納した PDF の内容で Q/A をする

ユーザーが質問した質問に対して、Vertex AI Search の検索で取得した類似のテキスト情報の中から回答を出すようにします。この仕組みを簡単に実現するフレームワークとして LangChain の RetrievalQA を利用します。

In [ ]:
# Create chain to answer questions
from langchain.chains import RetrievalQA

# Uses LLM to synthesize results from the search index.
# We use Gemini API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

ここで質問を定義します。Vertex AI Search に登録した文書にしたがって適宜修正してください。

In [ ]:
#query = input("Enter query:")
query = "Cloud Spannerの特徴は？" # @param {type:"string"}


LLM に直接問い合わせをした場合は、一般の知識に基づいて回答します。その内容を確認します。

In [ ]:
llm.invoke(query)

Vertex AI Search に登録した PDF からの回答はどのようになるでしょう？

In [ ]:
response = qa.invoke(query)
response["result"]

回答内容を作成したソースを確認します。

In [ ]:
response["source_documents"]

どうして PDF の内容をもとに回答しているかを理解するには、上で利用した LangChain の RetrievalQA の `stuff` タイプのソースコードを参照するのが分かりやすいです。

https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/question_answering/stuff_prompt.py

```
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
```
訳
```
次のコンテキストをもとに、最後の質問に答えてください。もし答えが分からない場合は、分からないと答えてください。勝手に答えをでっち上げないでください。
```

上記のように、最終の回答はプロンプト エンジニアリングで、正確な情報ソースをプロンプトにコンテキスト情報として渡し、そのコンテキスト情報の範囲内で回答するように LLM に依頼しています。



## LangChain Expression Language で改良
ここでは、上記と同じ処理を LangChain Expression Language (LCEL) で実装した方式に改良します。

In [ ]:
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

template = """次のコンテキスト情報を利用して、最後の質問に答えてください。回答は300字程度で回答してください。:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | RunnableParallel({
      "result": prompt | llm,
      "source_documents": itemgetter("context"),
    })
)

In [ ]:
chain.invoke(query)

以上、ありがとうございました。

## 参考情報
- [Question Answering Over Documents](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/retrieval-augmented-generation/examples/question_answering.ipynb)
- [LangChain](https://python.langchain.com/docs/get_started/introduction.html)
- [Overview of Generative AI on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)